### Соединение таблицы лизинголучателей, лизингодателей, предметов лизинга, определить записи по одному договору по полю origin_by_message
Входных данных нет  
Чтение всех таблиц, соединение таблиц лизинголучателей, лизингодателей, предметов лизинга, определение записией по одному договору по полю origin_by_message  
Merge and clean  
Выходной файл data/interm/merge_cleaned.pkl  

<a id="Initialization"></a>

# Инициализация

In [1]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки
%run ../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init_mdm.ipynb

In [2]:
# сюда таблицы будем кидать, для отчетов
name = pre + "data/interm/auxilary/merge_and_clean.xlsx"
writer = ExcelWriter(name)

In [3]:
name = pre + "data/interm/external_pkl/financialleasecontract.pkl"
financialleasecontract = pd.read_pickle(name, compression={'method':'zip'})

tmp = financialleasecontract[financialleasecontract['new_kreditmessage_number']=='02921408']
tmp.to_excel(writer, sheet_name='financialleasecontract')

print("Прочитанный размер файла из financialleasecontract {}".format(financialleasecontract.shape))
# financialleasecontract.head(1)

Прочитанный размер файла из financialleasecontract (1416028, 20)


In [4]:
name = pre + "data/interm/external_pkl/financialleasecontractlessees.pkl"
financialleasecontractlessees = pd.read_pickle(name, compression={'method':'zip'})

print("Прочитанный размер файла из financialleasecontractlessees {}".format(financialleasecontractlessees.shape))
# financialleasecontractlessees.head(1)

Прочитанный размер файла из financialleasecontractlessees (1411919, 12)


In [5]:
name = pre + "data/interm/external_pkl/financialleasecontractlessors.pkl"
financialleasecontractlessors = pd.read_pickle(name, compression={'method':'zip'})

print("Прочитанный размер файла из financialleasecontractlessors {}".format(financialleasecontractlessors.shape))
# financialleasecontractlessors.head(1)

Прочитанный размер файла из financialleasecontractlessors (1341360, 12)


In [6]:
name = pre + "data/interm/external_pkl/financialleasecontractobject.pkl"
financialleasecontractobject = pd.read_pickle(name, compression={'method':'zip'})

print("Прочитанный размер файла из financialleasecontractobject {}".format(financialleasecontractobject.shape))
# financialleasecontractobject.head(1)

Прочитанный размер файла из financialleasecontractobject (3210319, 12)


In [7]:
name = pre + "data/interm/external_pkl/Классификатор имущества.pkl"
df_codes = pd.read_pickle(name, compression={'method':'zip'})

print("Прочитанный размер файла из Классификатор имущества {}".format(df_codes.shape))
# df_codes.head(1)

Прочитанный размер файла из Классификатор имущества (168, 2)


## financialleasecontract

In [8]:
# определяем дубликаты по New_kreditmessage_number
num, duplicates = identify_duplicates(financialleasecontract, 'new_kreditmessage_number')
Check_dupl(num, duplicates)

num, duplicates = identify_duplicates(financialleasecontract, 'Id')
Check_dupl(num, duplicates)

In [9]:
financialleasecontract['new_leasing_enddate'] = pd.to_datetime(financialleasecontract['new_leasing_enddate'], errors='coerce')
financialleasecontract['leasingcontract_year'] = financialleasecontract['new_leasingcontract_date'].dt.year
financialleasecontract['leasing_startyear'] = financialleasecontract['new_leasing_startdate'].dt.year

financialleasecontract['Id'] = financialleasecontract['Id'].astype(str)

financialleasecontract['new_kreditmessage_type'] = financialleasecontract['new_kreditmessage_type'].astype('category')


## financialleasecontractobject

In [10]:
financialleasecontractobject.rename(columns={'New_kreditmessage_number':'new_kreditmessage_number'}, inplace=True)

In [11]:
# определяем дубликаты по Id
num, duplicates = identify_duplicates(financialleasecontractobject, 'Id')
Check_dupl(num, duplicates)

In [12]:
financialleasecontractobject['Id'] = financialleasecontractobject['Id'].astype(str)

In [13]:
# определяем дубликаты по New_kreditmessage_number
num, duplicates = identify_duplicates(financialleasecontractobject, 'new_kreditmessage_number')
# Check_dupl(num, duplicates)
# вывод на 1 договор несколько ПЛ

In [14]:
financialleasecontractobject['new_leasingsubject_code'] = [code if code =='99' or  code =='' or code[0] =='0' else  "0" + code for code in financialleasecontractobject['new_leasingsubject_code']]

In [15]:
objects = pd.merge(financialleasecontractobject, df_codes, how='left', left_on='new_leasingsubject_code', right_on='content_FinancialLeaseSubject_ClassifierCode')

In [16]:
objects.loc[objects['content_FinancialLeaseSubject_ClassifierCode'].isnull(), 'ClassifierName'] = objects.loc[objects['content_FinancialLeaseSubject_ClassifierCode'].isnull(), 'new_leasingsubject'] 


In [17]:
objects.drop(columns=['new_leasedassetsclass', 'content_FinancialLeaseSubject_ClassifierCode'], inplace=True)

удалим дубли по предметам лизинга, оставляя приоритет транспортные средства

In [18]:
"""  если автомобиль, то пусть будет самым приоритетным, потом идет транспортное средство, потом все остальное.
 то оно пусть будет в приоритете, добавим prefix 0, 1. к остальным кодам '2'"""
 
def car_priority(ClassifierCode):
   prefix = np.where(ClassifierCode.isin(['0106008', '0106011']), '0', np.where(ClassifierCode.str.startswith('0106'), '1', '2')) 
   return prefix + ClassifierCode

sort_object = objects.sort_values(by = ['new_leasingsubject_code'], key=car_priority)
# sort_object[sort_object['new_kreditmessage_number']=='01025531']

In [19]:
object_one_pl = sort_object.drop_duplicates(subset='new_kreditmessage_number', keep='first').reset_index(drop=True)

## financialleasecontractlessees (лизингополучатели)

In [20]:
# определяем дубликаты по New_kreditmessage_number
num, duplicates = identify_duplicates(financialleasecontractlessees, 'new_kreditmessage_number')
print(duplicates['new_lessee_inn'].nunique())
Check_dupl(num, duplicates, display=False)
# вывод на 1 договор несколько лизингополучателей, таких 339 различных ИНН, похоже что входят в группу

405
 Ошибка. Количество дубликатов 1376


Для дальнейшего анализа дубли удалять не будем, будем считать за 2 договора или наоборот удалить, чтобы не мешались

In [21]:
financialleasecontractlessees.drop_duplicates(subset='new_kreditmessage_number', inplace=True, ignore_index=True)

In [22]:
inn_lengh = financialleasecontractlessees['new_lessee_inn'].str.len()
print(inn_lengh.unique() )                    
inn_lengh.value_counts()

[12. 10. nan]


10.0    1235543
12.0     173516
Name: new_lessee_inn, dtype: int64

In [23]:
financialleasecontractlessees['Id'] = financialleasecontractlessees['Id'].astype(str)

## financialleasecontractlessors

In [24]:
# определяем дубликаты по New_kreditmessage_number
num, duplicates = identify_duplicates(financialleasecontractlessors, 'new_kreditmessage_number')
print(duplicates['new_creditor_inn'].nunique())
Check_dupl(num, duplicates, display=False)
# вывод на 1 договор несколько лизингодателей, но таких всего 8 ИНН, поэтому тоже удалю произвольным способом

8
 Ошибка. Количество дубликатов 43


In [25]:
financialleasecontractlessors.drop_duplicates(subset='new_kreditmessage_number', inplace=True, ignore_index=True)

In [26]:
financialleasecontractlessors['Id'] = financialleasecontractlessors['Id'].astype(str)

## находим цепочки по new_kreditmessage_number, определяем первичный договор

In [27]:
G = nx.from_pandas_edgelist(financialleasecontract[financialleasecontract['new_changemessage_publishe_number'].notnull()], \
                            'new_changemessage_publishe_number', 'new_kreditmessage_number', None, nx.DiGraph)

In [28]:
nodes = {node:nx.ancestors(G, node) for node in G.nodes}
origins = {node for node, ancestors in nodes.items() if len(ancestors)==0}
df_nodes = pd.DataFrame(nodes.items(), columns=['new_kreditmessage_number', 'ancestors'])
df_nodes['origin_by_message'] = [ row.new_kreditmessage_number if len(row.ancestors)==0 else (row.ancestors & origins).pop() for row in df_nodes.itertuples()]
df_nodes[(df_nodes['new_kreditmessage_number'].isin(['05639777', '12044325',  '12178446']))]

,new_kreditmessage_number,ancestors,origin_by_message
582749,05639777,{},05639777
582750,12044325,{05639777},05639777
587877,12178446,"{12044325, 05639777}",05639777


## Таблица договоров с определенным первичным приложением

In [29]:
financialleasecontract_origin = pd.merge(financialleasecontract, df_nodes, how='left', on='new_kreditmessage_number')
financialleasecontract_origin.loc[financialleasecontract_origin['origin_by_message'].isnull(), 'origin_by_message'] = \
        financialleasecontract_origin.loc[financialleasecontract_origin['origin_by_message'].isnull(), 'new_kreditmessage_number']

##  Определение, чем соединять договоры с таблицей лизингодателей

In [30]:
lessors_tmp = pd.merge(financialleasecontract_origin, financialleasecontractlessors[['Id', 'new_creditor_inn', 'new_kreditmessage_number']], how='left', 
                            on='new_kreditmessage_number', suffixes=["", "_lessor"])
lessors_tmp['Id_lessor'].fillna("", inplace=True)

Check_length(len(financialleasecontract_origin), len(lessors_tmp))

In [31]:
grouped = lessors_tmp.groupby(['origin_by_message'])
lessors_tmp['lessors_unique'] = grouped['new_creditor_inn'].transform('nunique')
lessors_tmp['Id_lessor_max'] = grouped['Id_lessor'].transform('max')
# get_contract_by_number(lessors_tmp, 'АЛ 192386/01-21')

In [32]:
# там где Id_lessor не заполнено, то проставим то, у кого нашли в цепочке, соединять будем уже по новому полю
mask = (lessors_tmp['Id_lessor']=="")
lessors_tmp.loc[mask, 'Id_lessor_to_merge'] = lessors_tmp.loc[mask, 'Id_lessor_max'] 
lessors_tmp.loc[~mask, 'Id_lessor_to_merge'] = lessors_tmp.loc[~mask, 'Id_lessor'] 
# get_contract_by_number(lessors_tmp, 'АЛ 192386/01-21')

In [33]:
# таким проставим одного из  лизингодателей 
mask = (lessors_tmp['lessors_unique']>1)
lessors_tmp.loc[mask, 'Id_lessor_to_merge'] = lessors_tmp.loc[mask, 'Id_lessor_max'] 
# get_contract_by_number(lessors_tmp, '004/20Б-ЛФ')

In [34]:
# теперь оставим в таблице контрактов только поле с которым будем соединять таблицу лизингополучателей
contract_lessors_to_merge = lessors_tmp.drop(columns=['Id_lessor', 'new_creditor_inn' ,'lessors_unique', 'Id_lessor_max'])


## Определение, чем соединять договоры с таблицей предметов лизинга

In [35]:
objects_tmp = pd.merge(contract_lessors_to_merge, object_one_pl[['Id', 'new_leasingsubject_guid', 'new_kreditmessage_number']], how='left', on='new_kreditmessage_number', suffixes=["", "_object"])\
                .sort_values(by='new_kreditmessage_date')
objects_tmp['Id_object'].fillna("", inplace=True)

Check_length(len(contract_lessors_to_merge), len(objects_tmp))

In [36]:
grouped = objects_tmp.groupby(['origin_by_message'])
objects_tmp['object_unique'] = grouped['new_leasingsubject_guid'].transform('nunique')
objects_tmp['Id_object_max'] = grouped['Id_object'].transform('max')


In [37]:
# там где Id_lessor не заполнено, то проставим то, у кого нашли в цепочке, соединять будем уже по новому полю
mask = (objects_tmp['Id_object']=="")
objects_tmp.loc[mask, 'Id_object_to_merge'] = objects_tmp.loc[mask, 'Id_object_max'] 
objects_tmp.loc[~mask, 'Id_object_to_merge'] = objects_tmp.loc[~mask, 'Id_object'] 


In [38]:
# теперь может соединять
contract_lessors_objects_to_merge = objects_tmp.drop(columns=['Id_object', 'new_leasingsubject_guid' ,'object_unique', 'Id_object_max'])

In [39]:
contract_lessors_objects_to_merge.shape

(1416028, 26)

## Определение, чем соединять договоры с таблицей лизингополучателей

In [40]:
lessee_tmp = pd.merge(contract_lessors_objects_to_merge, financialleasecontractlessees[['Id', 'new_lessee_inn', 'new_kreditmessage_number']], how='left', on='new_kreditmessage_number', suffixes=["", "_lessee"])\
                .sort_values(by='new_kreditmessage_date')
lessee_tmp['Id_lessee'].fillna("", inplace=True)

Check_length(len(lessee_tmp), len(contract_lessors_objects_to_merge))

grouped = lessee_tmp.groupby(['origin_by_message'])
lessee_tmp['lessee_unique'] = grouped['new_lessee_inn'].transform('nunique')
lessee_tmp['Id_lessee_max'] = grouped['Id_lessee'].transform('max')
lessee_tmp['Id_lessee_pre'] = grouped['Id_lessee'].shift(1)

In [41]:
""" там где Id_lessee не заполнено, то проставим то, у кого нашли в цепочке, соединять будем уже по новому полю,
но только если лизингополучатель один, если был перенайм, то не проставляем,
и то будут ошибки если не проставлено именно у перенайма

если запись окончания договора то проставляем лизингополучателя у последней записи
"""
mask_empty = (lessee_tmp['Id_lessee']=="") 
mask_unique = (lessee_tmp['lessee_unique']==1)
mask_several = (lessee_tmp['lessee_unique']>1) 
mask_stop = (lessee_tmp['new_kreditmessage_type']=='StopFinancialLeaseContract') 

lessee_tmp.loc[mask_empty & mask_unique, 'Id_lessee_to_merge'] = lessee_tmp.loc[mask_empty & mask_unique, 'Id_lessee_max'] 
lessee_tmp.loc[mask_empty & mask_several & mask_stop, 'Id_lessee_to_merge'] = lessee_tmp.loc[mask_empty & mask_several & mask_stop, 'Id_lessee_pre'] 

mask = (lessee_tmp['Id_lessee_to_merge'].isnull()) 
lessee_tmp.loc[mask, 'Id_lessee_to_merge'] = lessee_tmp.loc[mask, 'Id_lessee'] 

In [42]:
# теперь может соединять
contract_to_merge = lessee_tmp.drop(columns=['Id_lessee', 'new_lessee_inn' ,'lessee_unique', 'Id_lessee_max', 'Id_lessee_pre'])

## Теперь все соединяем

In [43]:
contracts_lessors = pd.merge(contract_to_merge, financialleasecontractlessors, how='left', left_on='Id_lessor_to_merge', right_on='Id', suffixes=["", "_lessor"])
Check_length(len(contracts_lessors), len(contract_to_merge))

In [44]:
contracts_lessee = pd.merge(contracts_lessors, financialleasecontractlessees, how='left', left_on='Id_lessee_to_merge', right_on='Id', suffixes=["", "_lessee"])

Check_length(len(contracts_lessee), len(contracts_lessors))

In [45]:
contracts_objects = pd.merge(contracts_lessee, financialleasecontractobject, how='left', left_on='Id_object_to_merge', right_on='Id', suffixes=["", "_object"])

Check_length(len(contracts_objects), len(contracts_lessee))

In [46]:
contracts = contracts_objects.sort_values(by='new_kreditmessage_date').reset_index()

#### удалим лишние строки и заполним null поля

1. удалим все строки где не заполнен лизингополучатель

In [47]:
n = contracts['new_lessee_inn'].isnull().sum()
print("лизингополучатель не заполнен у {} строк, {}%".format(n,round(n/len(contracts)*100, 2)))

лизингополучатель не заполнен у 3241 строк, 0.23%


In [48]:
contracts = contracts[contracts['new_lessee_inn'].notnull()]
print(contracts.shape)

(1412787, 64)


2. проставим пустым инн лизингодателя, будем считать что это компании коорые не должны раскрывать себя

In [49]:
n = contracts['new_creditor_inn'].isnull().sum()
print("лизингодатель не заполнен у {} строк, {}%".format(n,round(n/len(contracts)*100, 2)))

лизингодатель не заполнен у 395260 строк, 27.98%


In [50]:
contracts['new_creditor_inn'] = contracts['new_creditor_inn'].fillna("")

3. напишем что не найден предмет лизинга, если не найдено соответствие

In [51]:
n = contracts['new_leasingsubject'].isnull().sum()
print("не найден предмет лизинга не заполнен у {} строк, {}%".format(n,round(n/len(contracts)*100, 2)))

не найден предмет лизинга не заполнен у 9164 строк, 0.65%


In [52]:
contracts['new_leasingsubject'] = contracts['new_leasingsubject'].fillna("not_found")

4. определим причину завершения

In [53]:
def reason_to_parse(df):
     list_good = ['АКПП', 'ВЫУКП', 'ДКП', 'акт', 'вык4уп', 'выкуп', 'выполнен', '\\bдв\\b', 'досро', 'заверш', 'закрыт', \
               'исполнен', 'исполнил', 'истечение', 'купл.*прода', 'окончани', 'оплата по договору лизинга', 'план', \
               'по услови.*договор', 'погашен', 'полно', 'прекр', 'прекращен', 'продан', 'реализац', 'собственност', \
               'соответстви.*услови', 'срок']
     reason_good  =r'(?P<good>' + "|".join(i for i in list_good) + ')'

     list_bad = ['изъят', 'изят', 'нарушен', '\\bне.*выполне', '\\bне.*исполне', 'ненадлеж', \
               'неуплат.*лизинг',  'просрочк', 'рас.*гнут', 'растор', 'ржение', 'уведомле.*\\bраст', 'угон', 'утрат', 'хищен']
     reason_bad  =r'(?P<bad>' + "|".join(i for i in list_bad) + ')'     

     list_refusal = ['аванс.*\\bне', 'аннулирован', 'взаимн', 'дважд', 'договоренност', 'друго', 'дублир', 'задвое', 'инициатив', \
                    'коррект', '\\bне.*аванс', '\\bне.*верн', '\\bне.*вступ', '\\bне.*заключ', '\\bне.*подписан', '\\bне.*состо', '\\bне.*реализован',  \
                    'обоюдн', 'отказ', 'отклон',  'отмен', 'отсутств.*аванс', 'ошиб', 'повышение', 'поставщик', 'просрочен.*аванс', \
                    'решение.*лизингополучат', 'смена', 'сторн', 'требован.*клиент', 'удаление']
     reason_refusal  =r'(?P<refusal>' + "|".join(i for i in list_refusal) + ')'

     list_insurance = ['страхов']
     reason_insurance  =r'(?P<insurance>' + "|".join(i for i in list_insurance) + ')'

     list_cession = [ 'замен', 'перевод', 'перемен', 'перенайм', 'уступк']
     reason_cession  =r'(?P<cession>' + "|".join(i for i in list_cession) + ')'
     # 'цесси' убрала из списка, как как большинство идет как цессия с выкупом, не пойму почему не работает r'(?P<refusal>цесси(?!выкуп))'

     list_dead = ['гибел', 'тотал']
     reason_dead  =r'(?P<dead>' + "|".join(i for i in list_dead) + ')'

     list_unrecognized = ['возврат', 'возвращ', 'выбыт', '\\bисх', 'ресурс', 'чдт']
     reason_unrecognized  =r'(?P<unrecognized>' + "|".join(i for i in list_unrecognized) + ')'

     # #  какой то баран пишет "расторгнут (досрочный выкуп)" и понимай как знаешь. то ли тот удачно выкупил, то ли все же расторгли, но позволили ему выкупить. каким событием это является
     # # Расторжение в связи с выкупом
     # # расторжение в связи с досрочным выкупом ТС


     for pattern in [reason_good, reason_bad, reason_refusal, reason_insurance, reason_cession, reason_dead, reason_unrecognized]:
          df = df.join(df['new_terminationleasing_reason'].str.extract(pattern, re.IGNORECASE))
     
     mask_dead = (df['dead'].notnull())
     df.loc[mask_dead, 'reason'] = 'гибель/тотал'

     mask_insurance = (df['insurance'].notnull())
     df.loc[mask_dead & mask_insurance, 'reason'] = 'страховой случай (гибель/тотал)'

     mask_notclassified = (df['reason'].isnull())
     df.loc[mask_notclassified & mask_insurance, 'reason'] = 'страховой случай'

     mask_notclassified = (df['reason'].isnull())
     mask_refusal = (df['refusal'].notnull())
     df.loc[mask_notclassified & mask_refusal, 'reason'] = 'отмена'

     mask_notclassified = (df['reason'].isnull())
     mask_bad = (df['bad'].notnull())
     df.loc[mask_notclassified & mask_bad, 'reason'] = 'расторгнут'

     mask_notclassified = (df['reason'].isnull())
     mask_cession = (df['cession'].notnull())
     df.loc[mask_notclassified & mask_cession, 'reason'] = 'замена/перенайм/переуступка'

     mask_notclassified = (df['reason'].isnull())
     mask_good = (df['good'].notnull())
     df.loc[mask_notclassified & mask_good, 'reason'] = 'завершение'

     # df.drop(columns = ['good', 'bad', 'refusal', 'insurance', 'cession', 'dead', 'unrecognized'], inplace=True)
     
     return df


In [54]:
contracts['new_terminationleasing_reason'] = contracts['new_terminationleasing_reason'].fillna('')

In [55]:
tmp = contracts[contracts['new_terminationleasing_reason']!=""][['new_terminationleasing_reason', 'new_kreditmessage_number']].copy()
df_reasons = reason_to_parse(tmp)
# df_reasons.head()

In [56]:
contracts_w_reason = pd.merge(contracts, df_reasons[['reason', 'new_kreditmessage_number']], how='left', on = 'new_kreditmessage_number')

Check_length(len(contracts_w_reason), len(contracts))

# contracts_w_reason.head()


###  Определим у кого менялся ИНН лизингополучателя, значит была цессия

In [57]:
contracts_w_reason = contracts_w_reason.sort_values(by='new_kreditmessage_date').reset_index(drop=True)

In [58]:

contracts_w_reason['new_lessee_inn_next'] = contracts_w_reason.groupby(['origin_by_message'])['new_lessee_inn'].shift(-1)
contracts_w_reason['new_kreditmessage_date_next'] = contracts_w_reason.groupby(['origin_by_message'])['new_kreditmessage_date'].shift(-1)


In [59]:
tmp = contracts_w_reason[(contracts_w_reason['new_lessee_inn_next']!=contracts_w_reason['new_lessee_inn']) & (contracts_w_reason['new_lessee_inn_next'].notnull())]
print(tmp.shape)
# tmp.head()

(10899, 67)


In [60]:
#  проставим причину завершение договора 
mask_empty_reason = (contracts_w_reason['reason'].isnull())
mask_cession = (contracts_w_reason['new_lessee_inn_next']!=contracts_w_reason['new_lessee_inn']) & (contracts_w_reason['new_lessee_inn_next'].notnull())

contracts_w_reason.loc[mask_empty_reason & mask_cession, 'reason'] = 'определена цессия'
contracts_w_reason.loc[mask_empty_reason & mask_cession, 'cession_date'] = contracts_w_reason.loc[mask_empty_reason & mask_cession, 'new_kreditmessage_date_next']

contracts_w_reason['reason'] = contracts_w_reason['reason'].fillna("")

возьмем только нужные столбцы

In [61]:
keep_columns = ['new_kreditmessage_number', 'new_kreditmessage_date', 'new_kreditmessage_type', 
                     'new_leasingcontract_number', 'new_leasingcontract_date', 
                     'new_leasing_startdate', 'new_leasing_enddate', 
                     'new_sublease_agreement_sign', 
                     'new_changemessage_publishe_number', 
                     'new_terminationleasing_date', 'new_terminationleasing_reason', 'new_contract_status', 
                     'new_creditor_type', 'new_creditor_inn', 'new_creditor_ogrn', 
                     'new_creditor_name', 'new_lessee_type', 'new_lessee_inn',
                     'new_lessee_ogrn', 'new_lessee_name',  
                     'new_leasingsubjectid', 'new_leasingsubject_code', 'new_leasedassetsclass', 
                     'new_leasingsubject', 'origin_by_message', 'ancestors', 'reason', 'cession_date']
                     
contracts_cleaned = contracts_w_reason[keep_columns]
# contracts_cleaned.head()

In [62]:
contracts_cleaned.to_pickle(pre+"data/interm/merge_cleaned.pkl", compression={'method':'zip'})

In [63]:
contracts_cleaned.shape

(1412787, 28)

In [64]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save()

скорее всего файлы не понадобятся больше, поэтому их можно удалить с целью экономии рабочего места

In [65]:
need_to_remove_auxilary_files = True
if need_to_remove_auxilary_files:
    try:
        name = pre + "data/interm/external_pkl/financialleasecontract.pkl"
        if os.path.isfile(name):
            os.remove(name)
        name = pre + "data/interm/external_pkl/financialleasecontractlessees.pkl"
        if os.path.isfile(name):
            os.remove(name)
        name = pre + "data/interm/external_pkl/financialleasecontractlessors.pkl"
        if os.path.isfile(name):
            os.remove(name)
        name = pre + "data/interm/external_pkl/financialleasecontractobject.pkl"
        if os.path.isfile(name):
            os.remove(name)
    except:
        pass